In [ ]:
%matplotlib notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.pyplot as plt
import numpy as np
from plyfile import PlyData, PlyElement
import math

import sys
sys.path.append('../')
import dyrect as dy
from dyrect import draw_complex, unit_circle_sample, EpsilonNet, \
PatchedWitnessComplex, WitnessComplex

out_path = 'meshes/'

In [ ]:
def save_as_plyfile(pts, lines, triangles, mesh_name):
    x, y, z = pts[:, 0], pts[:, 1], pts[:, 2]
    pts = list(zip(x, y, z))

    # the vertex are required to a 1-d list
    vertex = np.array(pts, dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')])
    
    
    faces = np.array([(list(tr), 122, 122,122) for tr in triangles],
                     dtype=[('vertex_indices', 'i4', (3,)), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])
    edges = np.array([(list(ln), 50, 50, 50) for ln in lines],
                     dtype=[('vertex_indices', 'i4', (2,)), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])


    el0 = PlyElement.describe(vertex, 'vertex')
    el1 = PlyElement.describe(edges, 'edges')
    el2 = PlyElement.describe(faces, 'face')
    PlyData([el0, el1, el2], text=mesh_name).write(out_path + mesh_name + '.ply')


In [ ]:
def load_plyfile(filename):
    plydata = PlyData.read(filename)
    coords = plydata.elements[0].data
    simplices = {}
    simplices[0] = [(v,) for v in range(len(coords))]
    simplices[1] = [tuple(e) for [e,_,_,_] in plydata.elements[1].data]
    simplices[2] = [tuple(t) for [t,_,_,_] in plydata.elements[2].data]
    return coords, simplices

from itertools import combinations
def load_native_plyfile(filename):
    plydata = PlyData.read(filename)
    coords = plydata.elements[0].data
    simplices = {}
    simplices[0] = [(v,) for v in range(len(coords))]
    simplices[1] = []
    simplices[2] = [tuple(t) for [t] in plydata.elements[1].data]
    for s in simplices[2]:
        for edge in combinations(s, 2):
            if edge not in simplices[1]:
                simplices[1].append(edge)
    return coords, simplices


In [ ]:
# plydata = PlyData.read(out_path + "thomas_30_after_patching_level_3.ply")
plydata = PlyData.read(out_path + "torus_after_patching_level_2.ply")

coords, simplices = load_plyfile(out_path + "torus_after_patching_level_2.ply")
simplices

In [ ]:
plydata = PlyData.read(out_path + "lorenz_model_holes.ply")
plydata.elements[1].data

In [ ]:
coords, simplices = load_native_plyfile(out_path + "lorenz_model_holes.ply")

In [ ]:
simplices[2]

# UNIT SQUARE

In [ ]:
np.random.seed(2)
points = np.random.random((10000,2))
eps=0.055

EN = EpsilonNet(eps, 0)
EN.fit(points)
lms = EN.landmarks
print("Number of points: ", len(points))
print("Number of landmarks: ", len(lms))
# np.random.seed(2)
# points = np.random.random((24000,2))

fwidth = 12
fig = plt.figure(figsize=(fwidth, fwidth*0.4))
rows = 1
cols = 2

ax = plt.subplot(rows, cols, 1)
plt.scatter(points[:,0], points[:,1], s=0.2)
plt.scatter(lms[:,0], lms[:,1], s=5.2)

for lm in lms:
    crc = plt.Circle(lm, eps, color='r', alpha=0.05)
    ax.add_patch(crc)

ax = plt.subplot(rows, cols, 2)
# pwc = PatchedWitnessComplex(lms, points, 2, patching_level=1)
wc = WitnessComplex(lms, points, 2)
draw_complex(wc, fig=fig, ax=ax, vlabels=False)
# print(pwc._unproductive_witnesses)
[unprod_x, unprod_y] = np.array([points[i, :] for i in wc._barren_witnesses[2]]).T
ax.scatter(unprod_x, unprod_y, color='k', s = 0.5)
plt.show()

print("Betti numbers before patching: ", wc.betti_numbers)

#### patching
# patching_levels = [1,2,3,4]
patching_levels = [1,2,3]

fwidth = 20
fig = plt.figure(figsize=(fwidth, fwidth * 1./len(patching_levels)))
rows = 1
cols = len(patching_levels)

for ip, p in enumerate(patching_levels):
    ax = plt.subplot(rows, cols, ip+1)
    pwc2 = PatchedWitnessComplex(lms, points, 2, patching_level=p)
#     pwc2 = PatchedWitnessComplex(lms, eps, 2, points=points, patching=True, patching_level=p)
    print("Betti numbers at patching level ", p,  ": ", pwc2.betti_numbers)
    draw_complex(pwc2, fig=fig, ax=ax)
    plt.show()


# DOUBLE-CIRCLE

In [ ]:
np.random.seed(2)
crc1 = unit_circle_sample(3000, 0.75) + [1.1,0]
crc2 = unit_circle_sample(2000, 0.75) - [1.1,0]
points = np.append(crc1, crc2, axis=0)
eps=.2

EN = EpsilonNet(eps, 0)
EN.fit(points)
lms = EN.landmarks
print("Number of points: ", len(points))
print("Number of landmarks: ", len(lms))

fwidth = 12
fig = plt.figure(figsize=(fwidth, fwidth*0.4))
rows = 1
cols = 2

ax = plt.subplot(rows, cols, 1)
plt.scatter(points[:,0], points[:,1], s=0.2)
plt.scatter(lms[:,0], lms[:,1], s=5.2)

for lm in lms:
    crc = plt.Circle(lm, eps, color='r', alpha=0.05)
    ax.add_patch(crc)

ax = plt.subplot(rows, cols, 2)
wc = WitnessComplex(lms, points, 2)
draw_complex(wc, fig=fig, ax=ax, vlabels=False)
[unprod_x, unprod_y] = np.array([points[i, :] for i in wc._barren_witnesses[2]]).T
ax.scatter(unprod_x, unprod_y, color='k', s = 0.5)
plt.show()
print("Betti numbers before patching: ", wc.betti_numbers)

#### patching
patching_levels = [1,2]

fwidth = 20
fig = plt.figure(figsize=(fwidth, fwidth * 1./len(patching_levels)))
rows = 1
cols = len(patching_levels)

for ip, p in enumerate(patching_levels):
    ax = plt.subplot(rows, cols, ip+1)
    pwc2 = PatchedWitnessComplex(lms, points, 2, patching_level=p)
    print("Betti numbers at patching level ", p,  ": ", pwc2.betti_numbers)
    draw_complex(pwc2, fig=fig, ax=ax)
    plt.show()


# CUBE

In [ ]:
np.random.seed(2)
points = np.random.random((10000,3))

data_aspect = (np.ptp(points[:, 0]), np.ptp(points[:, 1]), np.ptp(points[:, 2]))
eps = 0.2
EN = EpsilonNet(eps, 0)
EN.fit(points)
lms = EN.landmarks
print("Number of points: ", len(points))
print("Number of landmarks: ", len(lms))

fwidth = 12
fig = plt.figure(figsize=(fwidth, fwidth*0.4))

rows = 1
cols = 2
ax = plt.subplot(rows, cols, 1, projection='3d')
# ax = fig.add_subplot(projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], s=1.5)
ax.scatter(lms[:,0], lms[:,1], lms[:,2], s=10.)
ax.set_box_aspect(data_aspect)

ax = plt.subplot(rows, cols, 2, projection='3d')
pwc = PatchedWitnessComplex(lms, eps, 3, points=points, patching=False, record_witnesses=True)
pwc._dim = 3
draw_complex(pwc, fig=fig, ax=ax, vlabels=True)
[unprod_x, unprod_y,  unprod_z] = np.array([points[i, :] for i in pwc._unproductive_witnesses[2]]).T
# ax.scatter(unprod_x, unprod_y, unprod_z, color='k', s = 0.5)
ax.set_box_aspect(data_aspect)
plt.show()
print("Betti numbers before patching: ", pwc.betti_numbers)
# save_as_plyfile(lms, pwc.simplices[1], pwc.simplices[2], 'torus_before_patching')

In [ ]:
#### PATCHING
patching_levels = [1,2,3,4,5,6,7,8]

fwidth = 20
fig = plt.figure(figsize=(fwidth, fwidth * 1./(len(patching_levels))))
rows = 1
cols = len(patching_levels)
# cols = int(np.ceil(len(patching_levels)/2))

for ip, p in enumerate(patching_levels):
    ax = plt.subplot(rows, cols, ip+1, projection='3d')
    pwc2 = PatchedWitnessComplex(lms, eps, 3, points=points, patching=True, patched_dimensions=[2,3], patching_level=p)
    pwc2._dim = 3
    print("Betti numbers at patching level ", p,  ": ", pwc2.betti_numbers)
    draw_complex(pwc2, fig=fig, ax=ax)
    ax.set_box_aspect(data_aspect)
    plt.show()

# TORUS

In [ ]:
np.random.seed(0)
points = dy.torus_sample(6000)
print(points.shape)
noise_level = 0.12
noise = np.random.random(points.shape) * noise_level - 0.5
points = points + noise

data_aspect = (np.ptp(points[:, 0]),np.ptp(points[:, 1]),np.ptp(points[:, 2]))
eps = 0.25
EN = EpsilonNet(eps, 0)
EN.fit(points)
lms = EN.landmarks
print("Number of points: ", len(points))
print("Number of landmarks: ", len(lms))

fig = plt.figure(figsize=(fwidth, fwidth*0.4))

rows = 1
cols = 2
ax = plt.subplot(rows, cols, 1, projection='3d')
# ax = fig.add_subplot(projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], s=1.5)
ax.scatter(lms[:,0], lms[:,1], lms[:,2], s=10.)
ax.set_box_aspect(data_aspect)

ax = plt.subplot(rows, cols, 2, projection='3d')
wc = WitnessComplex(lms, points, 2)
draw_complex(wc, fig=fig, ax=ax, vlabels=False)
[unprod_x, unprod_y,  unprod_z] = np.array([points[i, :] for i in wc._barren_witnesses[2]]).T
# ax.scatter(unprod_x, unprod_y, unprod_z, color='k', s = 0.5)
ax.set_box_aspect(data_aspect)
plt.show()
print("Betti numbers before patching: ", wc.betti_numbers)
save_as_plyfile(lms, wc.simplices[1], wc.simplices[2], 'torus_before_patching')

In [ ]:
#### PATCHING
patching_levels = [1,2,3]

fwidth = 20
fig = plt.figure(figsize=(fwidth, fwidth * 1./(len(patching_levels))))
rows = 1
cols = len(patching_levels)
# cols = int(np.ceil(len(patching_levels)/2))

for ip, p in enumerate(patching_levels):
    ax = plt.subplot(rows, cols, ip+1, projection='3d')
    pwc2 = PatchedWitnessComplex(lms, points, 2, patching_level=p)
    print("Betti numbers at patching level ", p,  ": ", pwc2.betti_numbers)
    draw_complex(pwc2, fig=fig, ax=ax)
    ax.set_box_aspect(data_aspect)
    plt.show()
    save_as_plyfile(lms, pwc2.simplices[1], pwc2.simplices[2], 'torus_after_patching_level_' + str(p))

# LORENZ ATTRACTOR

In [ ]:
np.random.seed(3)
points = dy.lorenz_attractor(10000)
data_aspect = (np.ptp(points[:, 0]),np.ptp(points[:, 1]),np.ptp(points[:, 2]))
eps = 2.
eps_str = str(int(eps*10))
EN = EpsilonNet(eps, 0)
EN.fit(points)
lms = EN.landmarks

np.random.seed(1)
points = dy.lorenz_attractor(200000)
print("Number of points: ", len(points))
print("Number of landmarks: ", len(lms))

fwidth = 12
fig = plt.figure(figsize=(fwidth, fwidth*0.4))

rows = 1
cols = 2
ax = plt.subplot(rows, cols, 1, projection='3d')
# ax = fig.add_subplot(projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], s=1.5)
ax.scatter(lms[:,0], lms[:,1], lms[:,2], s=10.)
ax.set_box_aspect(data_aspect)

ax = plt.subplot(rows, cols, 2, projection='3d')
wc = WitnessComplex(lms, points, 3)
print("Betti numbers before patching: ", wc.betti_numbers)
draw_complex(wc, fig=fig, ax=ax, vlabels=False)
[unprod_x, unprod_y,  unprod_z] = np.array([points[i, :] for i in wc._barren_witnesses[2]]).T
# pwc = PatchedWitnessComplex(lms, eps, 3, points=points, patching=False, record_witnesses=True)
# pwc._dim = 3
# draw_complex(pwc, fig=fig, ax=ax, vlabels=True)
# [unprod_x, unprod_y,  unprod_z] = np.array([points[i, :] for i in pwc._unproductive_witnesses[2]]).T
# ax.scatter(unprod_x, unprod_y, unprod_z, color='k', s = 0.5)
ax.set_box_aspect(data_aspect)
plt.show()
save_as_plyfile(lms, wc.simplices[1], wc.simplices[2], 'lorenz_' + eps_str + '_before_patching')

In [ ]:
#### PATCHING
patching_levels = [1,2,3]

fwidth = 12
fig = plt.figure(figsize=(fwidth, fwidth * 1./(len(patching_levels))))
rows = 1
cols = len(patching_levels)
# cols = int(np.ceil(len(patching_levels)/2))

for ip, p in enumerate(patching_levels):
    ax = plt.subplot(rows, cols, ip+1, projection='3d')
    pwc2 = PatchedWitnessComplex(lms, points, 3, patching_level=p)
    print("Betti numbers at patching level ", p,  ": ", pwc2.betti_numbers)
    draw_complex(pwc2, fig=fig, ax=ax)
    ax.set_box_aspect(data_aspect)
    plt.show()
    save_as_plyfile(lms, pwc2.simplices[1], pwc2.simplices[2], 'lorenz_' + eps_str + '_after_patching_level_' + str(p))

# Thomas ATTRACTOR

In [ ]:
np.random.seed(3)
# # points = dy.thomas_attractor(10000, step=0.001, starting_point=[0.1, 0.1, 0.1])
points = dy.generate_points(dy.thomas_eq, 3, [1.1, 1.1, -0.01], 40000, int_step = 0.042)
points2 = dy.generate_points(dy.thomas_eq, 3, [-0.2, 1.1, -0.01], 40000, int_step = 0.042)
points3 = dy.generate_points(dy.thomas_eq, 3, [0.5, 0.5, -0.01], 40000, int_step = 0.042)

points = np.vstack((points, points2, points3))
# points = points
print(points.shape)

data_aspect = (np.ptp(points[:, 0]),np.ptp(points[:, 1]),np.ptp(points[:, 2]))
eps = .25
eps_str = str(int(eps*100))
EN = EpsilonNet(eps, 0)
EN.fit(points)
lms = EN.landmarks

# np.random.seed(1)
# points = dy.lorenz_attractor(200000)
print("Number of points: ", len(points))
print("Number of landmarks: ", len(lms))
print(np.max(lms[:,0]), np.max(lms[:,1]))

print(points[:,:10])
fwidth = 12
fig = plt.figure(figsize=(fwidth, fwidth*0.4))

rows = 1
cols = 2
ax = plt.subplot(rows, cols, 1, projection='3d')
# ax = fig.add_subplot(projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], s=1.5)
ax.scatter(lms[:,0], lms[:,1], lms[:,2], s=10.)
ax.set_box_aspect(data_aspect)

In [ ]:
filenamebase = 'thomas'

ax = plt.subplot(rows, cols, 2, projection='3d')
wc = WitnessComplex(lms, points, 3)
print("Betti numbers before patching: ", wc.betti_numbers)
draw_complex(wc, fig=fig, ax=ax, vlabels=False)
[unprod_x, unprod_y,  unprod_z] = np.array([points[i, :] for i in wc._barren_witnesses[2]]).T
ax.set_box_aspect(data_aspect)
plt.show()
save_as_plyfile(lms, wc.simplices[1], wc.simplices[2], filenamebase + '_' + eps_str + '_before_patching')

#### PATCHING
patching_levels = [4]
max_patched_dimensions = 2

fwidth = 12
fig = plt.figure(figsize=(fwidth, fwidth * 1./(len(patching_levels))))
rows = 1
cols = len(patching_levels)
# cols = int(np.ceil(len(patching_levels)/2))

for ip, p in enumerate(patching_levels):
    ax = plt.subplot(rows, cols, ip+1, projection='3d')
    pwc2 = PatchedWitnessComplex(lms, points, 3, patching_level=p, max_patched_dimensions=max_patched_dimensions)
    print("Betti numbers at patching level ", p,  ": ", pwc2.betti_numbers)
    draw_complex(pwc2, fig=fig, ax=ax)
    ax.set_box_aspect(data_aspect)
    plt.show()
    save_as_plyfile(lms, pwc2.simplices[1], pwc2.simplices[2], filenamebase + '_' + eps_str + '_after_patching_level_' + str(p))

# KLEIN BOTTLE

In [ ]:
def klein_bottle(n=2000):
    """
    @param n:
    """
    # parameters of the klein bottle
    kr = 4.
    kp = 6.
    ke = 1. / 2.

    data = {}
    # print(sp, f_label(rot))
#     tor = dy.torus_rotation_interval(n, steps=[[np.sqrt(2),np.sqrt(3)]], starting_point=[0.0,0.0]) * 2 * np.pi
    sq = np.random.random((n, 2)) * 2 * np.pi
#     print(sq)
    data= sq
    A = np.array([[1 / 4., 1 / 4., 1 / 4., 1 / 4.], 
                  [1 / 4., 1 / 4., 1 / 4., -1 / 4.],
                  [1 / 4., 1 / 4., -1 / 4., -1 / 4.], 
                  [1 / 4., -1 / 4., -1 / 4., -1 / 4.]])
    cos0d2 = np.cos(sq[:, 0] / 2.)
    sin0d2 = np.sin(sq[:, 0] / 2.)
    cos0 = np.cos(sq[:, 0])
    sin0 = np.sin(sq[:, 0])
    sin1 = np.sin(sq[:, 1])
    cos1 = np.cos(sq[:, 1])
    sin1m2 = np.sin(sq[:, 1] * 2.)
    klein = np.array([kr * (cos0d2 * cos1 - sin0d2 * sin1m2),
        kr * (sin0d2 * cos1 - cos0d2 * sin1m2),
        kp * cos0 * (1 + ke * sin1),
        kp * sin0 * (1 + ke * sin1)]).transpose().reshape((n, 4))
    shifted_klein = np.dot(A, klein.transpose()).transpose()
#     data[('klein', f_label(sp), f_label(rot[0]), f_label(rot[1]))] = shifted_klein
#     for d in dims:
#         emb = dy.embedding(shifted_klein[:, 0].reshape((n,)), d, delay)
#         data[('emb', 0, d, f_label(sp), f_label(rot[0]), f_label(rot[1]))] = emb
    return klein

np.random.seed(0)
pts = np.array(klein_bottle(10000))
print(pts.shape)
fig = plt.figure()
ax = plt.subplot(111, projection='3d')
ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=0.5)

In [ ]:
np.random.seed(0)
points = klein_bottle(20000)

data_aspect = (np.ptp(points[:, 0]),np.ptp(points[:, 1]),np.ptp(points[:, 2]))
eps = 1.
EN = EpsilonNet(eps, 0)
EN.fit(points)
lms = EN.landmarks
print("Number of points: ", len(points))
print("Number of landmarks: ", len(lms))

fwidth=12
fig = plt.figure(figsize=(fwidth, fwidth*0.4))

rows = 1
cols = 2
ax = plt.subplot(rows, cols, 1, projection='3d')
# ax = fig.add_subplot(projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], s=1.5)
ax.scatter(lms[:,0], lms[:,1], lms[:,2], s=10.)
ax.set_box_aspect(data_aspect)

ax = plt.subplot(rows, cols, 2, projection='3d')
pwc = PatchedWitnessComplex(lms, eps, 2, points=points, patching=False, record_witnesses=True)
pwc._dim = 3
draw_complex(pwc, fig=fig, ax=ax, vlabels=True)
[unprod_x, unprod_y,  unprod_z, unprod_t] = np.array([points[i, :] for i in pwc._unproductive_witnesses[2]]).T
# ax.scatter(unprod_x, unprod_y, unprod_z, color='k', s = 0.5)
ax.set_box_aspect(data_aspect)
plt.show()
print("Betti numbers before patching: ", pwc.betti_numbers)
save_as_plyfile(lms, pwc.simplices[1], pwc.simplices[2], 'klein_before_patching')

In [ ]:
#### PATCHING
patching_levels = [1,2,3]

fwidth = 12
fig = plt.figure(figsize=(fwidth, fwidth * 1./(len(patching_levels))))
rows = 1
cols = len(patching_levels)
# cols = int(np.ceil(len(patching_levels)/2))

for ip, p in enumerate(patching_levels):
    ax = plt.subplot(rows, cols, ip+1, projection='3d')
    pwc2 = PatchedWitnessComplex(lms, eps, 3, points=points, patching=True, patching_level=p)
    pwc2._dim = 3
    print("Betti numbers at patching level ", p,  ": ", pwc2.betti_numbers)
    draw_complex(pwc2, fig=fig, ax=ax)
    ax.set_box_aspect(data_aspect)
    plt.show()
    save_as_plyfile(lms, pwc2.simplices[1], pwc2.simplices[2], 'klein_after_patching_level_' + str(p))

# STANFORD BUNNY

In [ ]:
plydata = PlyData.read('models/bunny/reconstruction/bun_zipper.ply')
px = plydata.elements[0].data['x']
py = plydata.elements[0].data['y']
pz = plydata.elements[0].data['z']
points = np.transpose(np.array([px, py, pz]))
points.shape

In [ ]:
data_aspect = (np.ptp(points[:, 0]),np.ptp(points[:, 1]),np.ptp(points[:, 2]))
eps = 0.01
EN = EpsilonNet(eps, 0)
EN.fit(points)
lms = EN.landmarks
print("Number of points: ", len(points))
print("Number of landmarks: ", len(lms))

fwidth = 12
fig = plt.figure(figsize=(fwidth, fwidth*0.4))

rows = 1
cols = 2
ax = plt.subplot(rows, cols, 1, projection='3d')
# ax = fig.add_subplot(projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], s=1.5)
ax.scatter(lms[:,0], lms[:,1], lms[:,2], s=10.)
ax.set_box_aspect(data_aspect)

ax = plt.subplot(rows, cols, 2, projection='3d')
pwc = PatchedWitnessComplex(lms, points, 2, patching_level=4, max_patched_dimensions=2)

# pwc._dim = 3
draw_complex(pwc, fig=fig, ax=ax, vlabels=True)
# [unprod_x, unprod_y,  unprod_z] = np.array([points[i, :] for i in pwc._unproductive_witnesses[2]]).T
# ax.scatter(unprod_x, unprod_y, unprod_z, color='k', s = 0.5)
ax.set_box_aspect(data_aspect)
plt.show()
# print("Betti numbers before patching: ", pwc.betti_numbers)
# save_as_plyfile(lms, pwc.simplices[1], pwc.simplices[2], 'bunny_before_patching')

In [ ]:
#### PATCHING
patching_levels = [1,2,3]

fwidth = 20
fig = plt.figure(figsize=(fwidth, fwidth * 1./(len(patching_levels))))
rows = 1
cols = len(patching_levels)
# cols = int(np.ceil(len(patching_levels)/2))

for ip, p in enumerate(patching_levels):
    ax = plt.subplot(rows, cols, ip+1, projection='3d')
    pwc2 = PatchedWitnessComplex(lms, eps, 3, points=points, patching=True, patching_level=p)
    pwc2._dim = 3
    print("Betti numbers at patching level ", p,  ": ", pwc2.betti_numbers)
    draw_complex(pwc2, fig=fig, ax=ax)
    ax.set_box_aspect(data_aspect)
    plt.show()
    save_as_plyfile(lms, pwc2.simplices[1], pwc2.simplices[2], 'bunny_after_patching_level_' + str(p))

# STANFORD ARMADILLO

In [ ]:
plydata = PlyData.read('models/armadillo/Armadillo.ply')
px = plydata.elements[0].data['x']
py = plydata.elements[0].data['y']
pz = plydata.elements[0].data['z']
points = np.transpose(np.array([px, py, pz]))
points.shape

In [ ]:
data_aspect = (np.ptp(points[:, 0]),np.ptp(points[:, 1]),np.ptp(points[:, 2]))
print(data_aspect)
eps = 5.5
EN = EpsilonNet(eps, 0)
EN.fit(points)
lms = EN.landmarks
print("Number of points: ", len(points))
print("Number of landmarks: ", len(lms))

In [ ]:
model_name = 'armadillo_5.5'

In [ ]:
fwidth = 12
fig = plt.figure(figsize=(fwidth, fwidth*0.4))

rows = 1
cols = 2
ax = plt.subplot(rows, cols, 1, projection='3d')
# ax = fig.add_subplot(projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], s=.01)
ax.scatter(lms[:,0], lms[:,1], lms[:,2], s=5.)
ax.set_box_aspect(data_aspect)

In [ ]:
ax = plt.subplot(rows, cols, 2, projection='3d')
pwc = PatchedWitnessComplex(lms, eps, 2, points=points, patching=False, record_witnesses=True)
pwc._dim = 3
draw_complex(pwc, fig=fig, ax=ax, vlabels=True)
[unprod_x, unprod_y,  unprod_z] = np.array([points[i, :] for i in pwc._unproductive_witnesses[2]]).T
# ax.scatter(unprod_x, unprod_y, unprod_z, color='k', s = 0.5)
ax.set_box_aspect(data_aspect)
plt.show()
print("Betti numbers before patching: ", pwc.betti_numbers)
save_as_plyfile(lms, pwc.simplices[1], pwc.simplices[2], model_name + '_before_patching')

In [ ]:
#### PATCHING
patching_levels = [1,2,3]

fwidth = 20
fig = plt.figure(figsize=(fwidth, fwidth * 1./(len(patching_levels))))
rows = 1
cols = len(patching_levels)
# cols = int(np.ceil(len(patching_levels)/2))

for ip, p in enumerate(patching_levels):
    ax = plt.subplot(rows, cols, ip+1, projection='3d')
    pwc2 = PatchedWitnessComplex(lms, eps, 2, points=points, patching=True, patching_level=p)
    pwc2._dim = 3
    print("Betti numbers at patching level ", p,  ": ", pwc2.betti_numbers)
    draw_complex(pwc2, fig=fig, ax=ax)
    ax.set_box_aspect(data_aspect)
    plt.show()
    save_as_plyfile(lms, pwc2.simplices[1], pwc2.simplices[2], model_name + '_after_patching_level_' + str(p))